In [1]:
import re
import pandas as pd
import spacy
import numpy as np

In [2]:
nlp = spacy.load('en', disable=["ner", "parser"])

In [3]:
def cleaning(doc):
    """
    :param doc: spacy Doc object processed by the pipeline
    :return: Text lemmatized and without stopwords
    """
    txt = [token.lemma_ for token in doc if not token.is_stop]

    # Since training with small document don't make great benefits, they are ignored.
    if len(txt) > 2:
        return ' '.join(txt)

In [4]:
from sqlitedict import SqliteDict

In [ ]:
categories = ["left", "left-center", "right", "Intellectual Dark Web", "Alt-lite"]

In [5]:
pua = SqliteDict(f"./../Sqlite/split_texts/{category}.sqlite", tablename="value", flag="r")

In [6]:
pua_clean = SqliteDict(f"{category}.sqlite_clean.sqlite", tablename="value", journal_mode="OFF")

In [7]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(value["text"])).lower() for value in pua.values())

In [8]:
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=32, n_threads=250)]

In [9]:
for text, id_c in zip(txt, pua.keys()):
    pua_clean[id_c] = {"text":text, "timestamp":pua[id_c]["timestamp"]}

In [10]:
pua_clean.commit()
pua_clean.close()

In [11]:
df_clean = pd.DataFrame({'clean': txt})

In [12]:
df_clean = df_clean.dropna().drop_duplicates()

In [14]:
df_clean.to_pickle(f"{category}.sqlite_clean.csv")